## Project - Airline AI Assistant

In [185]:
import os
import json
from dotenv import load_dotenv
import google.generativeai as genai
import gradio as gr
from IPython.display import Markdown, display, clear_output

In [186]:
load_dotenv()

google_api_key = os.getenv('GEMINI_API_KEY')

In [187]:
genai.configure(api_key=google_api_key)

In [188]:
system_message = "You are a helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers using markdown, no more than 1 sentence. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [189]:
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [190]:
price_function = genai.protos.FunctionDeclaration(
    name="get_ticket_price",
    description="Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    parameters=genai.protos.Schema(
        type=genai.protos.Type.OBJECT,
        properties={
            "destination_city": genai.protos.Schema(
                type=genai.protos.Type.STRING,
                description="The city that the customer wants to travel to"
            )
        },
        required=["destination_city"]
    )
)

In [191]:
def stream_gemini_with_history(message, history):
    
    model = genai.GenerativeModel(
        model_name="gemini-2.0-flash",
        system_instruction=system_message,
        tools=[price_function]
    )

    prompt = ""
    for user_msg, bot_msg in history:
        prompt += f"User: {user_msg}\nAssistant: {bot_msg}\n"
    prompt += f"User: {message}\nAssistant:"

    response = model.generate_content(prompt)

    if response.candidates[0].content.parts and hasattr(response.candidates[0].content.parts[0], "function_call"):
        all_parts = response.candidates[0].content.parts
        function_results = ""

        for part in all_parts:
            if hasattr(part, "function_call"):
                function_call = part.function_call
                function_name = function_call.name

                if function_call.args is not None:
                    function_args = dict(function_call.args)
                else:
                    function_args = {}

                print(f"Function called: {function_name} with args: {function_args}")

                if function_name == "get_ticket_price":
                    dest = function_args.get("destination_city")
                    if dest:
                        result = get_ticket_price(dest)
                        function_results += f"[Function call: {function_name}({function_args}) returned: {result}]\n"

        prompt += "\n" + function_results + "Assistant:"
        
    stream = model.generate_content(prompt, stream=True)

    response_text = ""
    for chunk in stream:
        if chunk.text:
            response_text += chunk.text
            yield history + [(message, response_text)], ""

In [192]:
def clear_chat():
    return [], ""

with gr.Blocks(theme=gr.themes.Soft()) as view:
    gr.Markdown("## 💬 FlightAI")

    chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)
    msg = gr.Textbox(label="Your message:", lines=2, placeholder="Type your message here...")

    with gr.Row():
        send_btn = gr.Button("Send")
        clear_btn = gr.Button("Clear Chat")

    msg.submit(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])
    send_btn.click(stream_gemini_with_history, inputs=[msg, chatbot], outputs=[chatbot, msg])

    clear_btn.click(clear_chat, outputs=[chatbot, msg])

view.launch()


C:\Users\chandula\AppData\Local\Temp\ipykernel_13916\1640575266.py:7: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot = gr.Chatbot(label="Conversation", show_copy_button=True)


* Running on local URL:  http://127.0.0.1:7872

To create a public link, set `share=True` in `launch()`.
